<a href="https://colab.research.google.com/github/phantomfait/RL_Lunar_Lander/blob/main/RL_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#@title Install dependencies (takes a few minutes)

!apt-get -qq update
!apt-get -qq install -y swig ffmpeg xvfb

!pip -q install --upgrade pip

# Gymnasium with Box2D (for LunarLander), Stable-Baselines3, and helpful video libs
!pip -q install "gymnasium[box2d]" "stable-baselines3" "sb3-contrib" "moviepy" "imageio[ffmpeg]" "numpy<2.0"


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
!pip install --upgrade --force-reinstall numpy==1.26.4
!pip install --upgrade --force-reinstall gymnasium==0.29.1


  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompati

  Using cached gymnasium-0.29.1-py3-none-any.whl.metadata (10 kB)
  Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
Using cached gymnasium-0.29.1-py3-none-any.whl (953 kB)
Using cached cloudpickle-3.1.1-py3-none-any.whl (20 kB)
Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
Using cached typing_extensions-4.14.1-py3-none-any.whl (43 kB)
  Attempting uninstall: farama-notifications
    Found existing installation: Farama-Notifications 0.0.4
    Uninstalling Farama-Notifications-0.0.4:
      Successfully uninstalled Farama-Notifications-0.0.4
  Attempting uninstall: typing-extensions
    Found existing in

In [2]:

#@title Imports & Versions
import os, sys, math, time, pathlib, random
import numpy as np

import gymnasium as gym
from gymnasium.wrappers import RecordVideo, RecordEpisodeStatistics

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecMonitor, DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

import matplotlib.pyplot as plt
from IPython.display import HTML, clear_output

print("Python:", sys.version)
import gymnasium
import stable_baselines3 as sb3
print("Gymnasium:", gymnasium.__version__)
print("Stable-Baselines3:", sb3.__version__)


Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Gymnasium: 1.2.0
Stable-Baselines3: 2.7.0


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [4]:

#@title Training Config
ENV_ID = "LunarLander-v3"  #Gymnasium's latest version
TOTAL_TIMESTEPS = 1_500_000

N_ENVS = 8                 # parallel envs for faster PPO training
SEED = 42

LOG_DIR = "./lander_logs"
VIDEO_DIR = "./lander_videos"
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(VIDEO_DIR, exist_ok=True)

np.random.seed(SEED)
random.seed(SEED)


In [5]:

#@title Make Environments (train & eval)

def make_single_env(seed_offset=0, render_mode=None):
    # For training, render_mode=None; for video, use "rgb_array"
    env = gym.make(ENV_ID, render_mode=render_mode)
    env.reset(seed=SEED + seed_offset)
    env = RecordEpisodeStatistics(env)
    return env

# Vectorized training envs
train_env = make_vec_env(
    lambda: make_single_env(render_mode=None),  # no rendering during training
    n_envs=N_ENVS,
    seed=SEED,
)
train_env = VecMonitor(train_env, filename=os.path.join(LOG_DIR, "monitor.csv"))

# Separate eval env (no vectorization required)
eval_env = make_single_env(render_mode=None)



/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

In [6]:

#@title 🧠 Create PPO model + callbacks

#stop early once reached a decent score
reward_threshold_callback = StopTrainingOnRewardThreshold(reward_threshold=300.0, verbose=1)

eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=LOG_DIR,
    log_path=LOG_DIR,
    eval_freq=max(10_000 // N_ENVS, 1),  # evaluate periodically
    n_eval_episodes=10,
    deterministic=True,
    render=False,
    callback_after_eval=reward_threshold_callback,
)

policy_kwargs = dict(net_arch=[256, 256])  # decent MLP for LunarLander

model = PPO(
    "MlpPolicy",
    train_env,
    verbose=1,
    tensorboard_log=LOG_DIR,
    seed=SEED,
    n_steps=2048,          # rollout length per env
    batch_size=256,        # minibatch size
    gae_lambda=0.95,
    gamma=0.999,
    n_epochs=10,
    learning_rate=3e-4,
    clip_range=0.2,
    policy_kwargs=policy_kwargs,
)
model


Using cuda device


/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [7]:

#@title Train PPO (this will take a while)
start = time.time()
model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=eval_callback, progress_bar=True)
wall = time.time() - start
print(f"Training finished in {wall/60:.1f} minutes.")

# Load best model (if saved), otherwise keep current
best_path = os.path.join(LOG_DIR, "best_model.zip")
if os.path.exists(best_path):
    print("Loading best model from", best_path)
    model = PPO.load(best_path, env=train_env, device="auto")


Logging to ./lander_logs/PPO_1


Output()

/usr/local/lib/python3.12/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/callbacks.py:418: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_monitor.VecMonitor object at 0x7e1a699bb6b0> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7e1a69932480>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation 
environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and 
rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(

...

Episode length: 242.50 +/- 16.05

------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 242          |
|    mean_reward          | 302          |
| time/                   |              |
|    total_timesteps      | 1500000      |
| train/                  |              |
|    approx_kl            | 0.0051825624 |
|    clip_fraction        | 0.0519       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.485       |
|    explained_variance   | 0.991        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.942        |
|    n_updates            | 910          |
|    policy_gradient_loss | 0.000429     |
|    value_loss           | 24.5         |
------------------------------------------


New best mean reward!

Stopping training because the mean reward 302.30  is above the threshold 300.0

Training finished in 17.3 minutes.
Loading best model from ./lander_logs/best_model.zip


In [10]:

#@title Evaluate policy
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=20, deterministic=True)
print(f"Mean reward over 20 eval episodes: {mean_reward:.2f} ± {std_reward:.2f}")


def run_episodes_and_success_rate(n_episodes=20):
    success = 0
    returns = []
    for ep in range(n_episodes):
        obs, info = eval_env.reset(seed=SEED + 100 + ep)
        done = False
        ep_ret = 0.0
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, info = eval_env.step(action)
            done = terminated or truncated
            ep_ret += reward
        returns.append(ep_ret)
        if ep_ret >= 200.0:
            success += 1
    return np.mean(returns), np.std(returns), success / n_episodes

m, s, succ = run_episodes_and_success_rate(20)
print(f"Episode return: {m:.1f} ± {s:.1f} | Success rate (≥200): {succ*100:.1f}%")


/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward over 20 eval episodes: 278.53 ± 18.68
Episode return: 291.3 ± 18.6 | Success rate (≥200): 100.0%


In [11]:
#@title Record a "successful" landing video (≥200 reward)
from pathlib import Path
from base64 import b64encode
from IPython.display import HTML
from stable_baselines3 import PPO

# Load best model from training
best_path = os.path.join(LOG_DIR, "best_model.zip")
assert os.path.exists(best_path), "Train first — no best_model.zip found!"
best_model = PPO.load(best_path, env=None, device="cpu")

max_attempts = 20   #  20 rollouts
success_video = None
success_return = None

for attempt in range(max_attempts):
    video_env = gym.make(ENV_ID, render_mode="rgb_array")
    prefix = f"best-success-ep{attempt}"
    video_env = RecordVideo(video_env, video_folder=VIDEO_DIR, name_prefix=prefix, episode_trigger=lambda e: True)

    obs, info = video_env.reset(seed=SEED + 1000 + attempt)
    done, ep_return = False, 0
    while not done:
        action, _ = best_model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = video_env.step(action)
        ep_return += reward
        done = terminated or truncated
    video_env.close()

    if ep_return >= 200:
        success_return = ep_return
        success_video = prefix
        print(f"✅ Found successful landing on attempt {attempt} with return {ep_return:.2f}")
        break

# If we found a success, display it
if success_video:
    mp4s = sorted(Path(VIDEO_DIR).glob(f"{success_video}*.mp4"), key=lambda p: p.stat().st_mtime, reverse=True)
    mp4_path = str(mp4s[0])
    mp4 = open(mp4_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML(f"<video width=600 controls autoplay loop src='{data_url}'></video>"))
else:
    print(f"❌ No successful landing found in {max_attempts} attempts.")


/usr/local/lib/python3.12/dist-packages/gymnasium/wrappers/rendering.py:296: UserWarning: WARN: Overwriting existing videos at /content/lander_videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


✅ Found successful landing on attempt 0 with return 305.07
